## Single Vending Machine Optimization Problem

## Problem Description

There are a few(12) vending machines at different locations. Each machine can  occupies 5 differnt kinds of drinks soda,cola,tropics, cherry, cola can etc. The goal here to maximize the minimum days of supply (dos) for each machine.

#### About Vending machine 
Input file - machines.csv

Each machine has unique machine id, machine_type, list of the product types that the machine can occupy, and supply per day(spd) of each product associated to the machine id.

#### About products
There are total 5 dictict types of product ids eg. [1:soda,2:cola,3:tropics,4:cherry,5:cola can]
each drinks has it's volumne.

#### Product machine
The volumn of the product decides how much aspece that product has in the vending machine.If vi


machine,product,spd


### Solution Approach
A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.
We now present a MIP formulation for the Vending Mcachine optimisation problem.

Decisions:

(1) Which products should go in wchine vending machine

(2) The ways columsn of machine has to fill

(3) Max(min(sum()))

## Model Formulation

### Sets 

$m \in M$: Set of Machines.{1,2,3,.....12}

$i \in I$: Set Machine Types, {1:closed_front_1, 2:closed_front_2, 3:closed_front_3, 4:closed_front_4, 5:closed_front_5, 6:closed_front_6}.

$c \in C$: Set of Columnsin a machine.{1:col1,2:col2,3:col3,4:col4,5:col5,6:col6,7:col7,8:col8,9:col9,10:col10,11:col11,12:col12}

$p \in P$: product {111,222,333,444,555}

$v \in V$: volume {12,16.9}




### Parameters

$cap_{m,i,c} \in \mathbb{R}^+$: Fixed caapcity associated with machine id,machine type,machine column for $m \in M$ of type $i \in I$ of column .

$spd_{m,i,p} \in \mathbb{R}^+$: Supply per day for a machine $m \in M$ and type $i \in I$ of product $p \in P$.

$vol_{p,v} \in \mathbb{R}^+$: Volume of product  $p \in P$ is $v \in V$. 



### Decision Variables

$select_{m,i,c,p}\in \mathbb{N}^+$: Quantity of product p (in units) that can be place in machine $m of type $i in column $c. 



### Objective Function

- **Total costs**. We want to  Maximize the minimum days of supply (dos) for each machine. 
maximise( min(capacity_product1=sum(product1)/spd_product1,...sum(product1) ))

\begin{equation}
\text{Max} \quad Z = \sum_{p \in P}_\cdot select_{d,t_initial,t_future}+\sum_{d \in D}  h_{d,t_initial,t_future}  \cdot select_{d,t_initial,t_future} + \sum_{d \in D} \sum_{i \in I} c_{d,i} \cdot x_{d,tstore,i}  \sum_{d \in D} \sum_{j \in J} d_{d,j} \cdot y_{d,tonline,j}
\tag{0}
\end{equation}

### Constraints

- **Column capacity constraint**. For each column  $c \in C$ ensure the products in each column is less then capacity .
\begin{equation}
\sum_{p \in P} select_{m,i,c,p} <= cap_{m,i,c}  \quad \forall i \in I \quad \forall tstore = {1: RDC, 3: HDC}
\tag{1}
\end{equation}

- **One column one product constraint**. For each online  $j \in J$ ensure that its demand is fulfilled.
\begin{equation}
\sum_{d \in D} y_{d,tonline,j} = dj \quad \forall j \in J \quad \forall tonline = {1: EDC, 3: HDC}
\tag{1}
\end{equation}

- **Ensure if distribution center is open before providing service to stores**. We need to ensure that we  only ship from DC $d \in D$,  if that dc has actually been built/open/ tranfer to type of DC that fulfills store's demand.

for B&M Stores
\begin{equation}
\sum_{i \in I} x_{d,tstore,i} \leq M \cdot select_{d,t_initial,t_future} \quad \forall d \in D
\tag{2}
\end{equation}

for online Stores
\begin{equation}
\sum_{j \in J} y_{d,tonline,j} \leq M \cdot select_{d,t_initial,t_future}  \quad \forall d \in D
\tag{2}
\end{equation}

In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
from gurobipy import multidict
from gurobipy import GRB
from gurobipy import min_

In [2]:
### Reading required files
capacity = pd.read_csv('capacity_matrix.csv')
items = pd.read_csv('items.csv')
machines = pd.read_csv('machines.csv')
columns_shelf=capacity.columns
columns_shelf[2:14]

## Handling Single Machine 
machines_single = machines[machines['machine_id']==1]

## Adding column_name optimions column to the machines 
machines_single=machines_single.merge(pd.DataFrame(columns_shelf[2:14].values), how='cross')
machines_single.rename(columns = {0:'Column'}, inplace = True)

In [4]:
items

,item_id,name,pack
0,111,soda,16.9oz bottle
1,222,cola,16.9oz bottle
2,333,tropics,16.9oz bottle
3,444,cherry,12oz can
4,555,cola can,12oz can


In [5]:
machines

,machine_id,machine_type,item_id,spd
0,1,closed_front_1,111,5.6
1,1,closed_front_1,222,5.4
2,1,closed_front_1,333,3.6
3,1,closed_front_1,444,1.5
4,1,closed_front_1,555,1.6
5,2,closed_front_2,111,5.6
6,2,closed_front_2,222,5.4
7,2,closed_front_2,333,3.6
8,2,closed_front_2,444,1.5
9,2,closed_front_2,555,1.6


In [5]:
## Data Correction- Not needed for single machine 
"""
machines2 = machines
correction = {'soda':111 , 'cola':222, 'tropics':333, 'cherry':444,'cola can':555}
machines=machines2.replace({"item_id": correction})
"""

'\nmachines2 = machines\ncorrection = {\'soda\':111 , \'cola\':222, \'tropics\':333, \'cherry\':444,\'cola can\':555}\nmachines=machines2.replace({"item_id": correction})\n'

In [237]:
#Sets-Single machine 
products = list(items['item_id'].unique())
machine = list(machines_single['machine_id'].unique())
machines_types = list(machines_single['machine_type'].unique())
columns = list(columns_shelf[2:14])#list(range(1,13)) ## conver to auto
volumns = list(capacity['pack'].unique())


In [238]:
#Single machine -Parameters -1
cap={}
for j in machines_types: 
    for k in volumns:
        for col in columns:
          column_name = col #'col'+str(col)
          cap[j,k,column_name]=capacity.loc[(capacity['machine_type']==j) & (capacity['pack']==k),column_name].values[0]
            

## Single Parameters -2
#𝑠𝑝𝑑𝑚,𝑖,𝑝∈ℝ+ : Supply per day for a machine 𝑚∈𝑀 and type 𝑖∈𝐼 of product 𝑝∈𝑃.
    
spdm = {}
spdm={j:k for j in zip(machines_single['machine_id'],machines_single['machine_type'],machines_single['item_id']) for k in machines.loc[(machines_single['machine_id']==j[0]) & (machines_single['machine_type']==j[1]) & (machines_single['item_id']==j[2]),'spd']}


## Parameters -3
#𝑣𝑜𝑙𝑝,𝑣∈ℝ+ : Volume of product  𝑝∈𝑃  is  𝑣∈𝑉 .
volp ={p:v for p in items['item_id'] for v in items.loc[items['item_id']==p,'pack'] }

                  

In [239]:
## Decision Variable -single machine 
select1 ={j:0 for j in zip(machines_single['machine_id'],machines_single['machine_type'],machines_single['item_id'],machines_single['Column'])}



In [240]:
combinations, scores = gp.multidict(select1)

In [96]:
#combinations

In [258]:
##Single mahine - Start model  create decision variable -

model = gp.Model('VendingMachineOptimization_singlemachine')
select_single = model.addVars(combinations, vtype=GRB.BINARY, name='select2')  #(1,'closed_front_2', '555', 1):12(number of products)
#select = model.addVars(combinations, obj=scores, name="prod_col")

In [54]:
## Single machine -objective value 
#obj = gp.quicksum(((select_single[m,i,p,c]*cap[i,volp[int(p)],c])/spdm[m,i,p]) for m,i,p,c in select1.keys())
#model.setObjective(obj, GRB.MAXIMIZE)


# Production capacity limits



In [243]:
## Building objective finction for descrete values 
M=len(products)
y = model.addVars(products, lb=-GRB.INFINITY, name="y")
#z = model.addVars(1, lb=-GRB.INFINITY, name="z")
z = model.addVar(vtype=GRB.CONTINUOUS, name="Z")
#u = model.addVars(M, name="u")

#model.addConstrs((y[p] == gp.quicksum(select_single[m,i,str(p),c]*cap[i,volp[int(p)],c] for m,i,g,c in select1.keys()) for p in products),name="set_y")
model.addConstrs((y[p] == gp.quicksum(select_single[m,i,str(p),c]*cap[i,volp[int(p)],c] for m,i,g,c in select1.keys()) 
                  for p in products),name="set_y")

#model.addConstr(min_(y,0),name="maxconstr")
#model.addConstrs((z[i] == min_(y[i]) for i in products), name="set_minobj")
model.addConstr(z == min_(y), name="min constraint")

model.setObjective(z, sense=GRB.MAXIMIZE)

#model.setObjective(min_(y,0), GRB.MAXIMIZE)
#m.addConstr(z == min_(x, y, constant=3))
#model.addConstrs((u[i] == abs_(z[i]) for i products)), name="set_u")
#model.addConstr(minobj == u.sum(), name="set_minobj")

In [8]:
## Modification in above 
y = model.addVars(products, lb=-GRB.INFINITY, name="y")
z = model.addVar(vtype=GRB.CONTINUOUS, name="Z")
model.addConstrs((y[p] == gp.quicksum(select_single[m,i,str(p),c]*cap[i,volp[int(p)],c] for m,i,g,c in select1.keys())/spdm[1,'closed_front_1',str(p)] 
                  for p in products),name="set_y")
model.addConstr(z == min_(y), name="min constraint")
model.setObjective(z, sense=GRB.MAXIMIZE)
               

NameError: name 'model' is not defined

In [260]:
y

{111: <gurobi.Var *Awaiting Model Update*>,
 222: <gurobi.Var *Awaiting Model Update*>,
 333: <gurobi.Var *Awaiting Model Update*>,
 444: <gurobi.Var *Awaiting Model Update*>,
 555: <gurobi.Var *Awaiting Model Update*>}

In [256]:
select_single

{(1,
  'closed_front_1',
  '111',
  'col1'): <gurobi.Var select2[1,closed_front_1,111,col1] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col2'): <gurobi.Var select2[1,closed_front_1,111,col2] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col3'): <gurobi.Var select2[1,closed_front_1,111,col3] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col4'): <gurobi.Var select2[1,closed_front_1,111,col4] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col5'): <gurobi.Var select2[1,closed_front_1,111,col5] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col6'): <gurobi.Var select2[1,closed_front_1,111,col6] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col7'): <gurobi.Var select2[1,closed_front_1,111,col7] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col8'): <gurobi.Var select2[1,closed_front_1,111,col8] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col9'): <gurobi.Var select2[1,closed_front_1,111,col9] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  '

In [227]:
""" # Reference
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("test")
x = m.addVars(5, vtype=GRB.CONTINUOUS, lb=0, ub=100, name="X")
z = m.addVar(vtype=GRB.CONTINUOUS, name="Z")
# Add constraint z = max(x1,x2,x3,x4,x5)
m.addConstr(z == gp.max_(x), name="max_contraint")
m.setObjective(z, sense=GRB.MINIMIZE)
"""

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-227-bcf227de1bbc>, line 10)

In [25]:
## Single machine - Constraint1 
"""
model.addConstrs((select_single[m,i,p,c]*cap[i,volp[int(p)],c] <= cap[i,volp[int(p)],c] for m,i,p,c in select_single.keys()),
                 name="Single_column_capacity")"""


'\nmodel.addConstrs((select_single[m,i,p,c]*cap[i,volp[int(p)],c] <= cap[i,volp[int(p)],c] for m,i,p,c in select_single.keys()),\n                 name="Single_column_capacity")'

In [262]:
## Single machine - Constraint2
model.addConstrs((select_single.sum(m,i,'*',c) <= 1
             for m,i,p,c in select_single.keys()),
            name="single_1_col_1prod") 


{(1, 'closed_front_1', '111', 'col1'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col2'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col3'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col4'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col5'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col6'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col7'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col8'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col9'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col10'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col11'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col12'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_

In [247]:
## Single machine Minimum demand has to be satisfied - Constraint2
model.addConstrs((select_single.sum(m,i,p,'*')*cap[i,volp[int(p)],c] >= spdm[m,i,p]
             for m,i,p,c in select_single.keys()),
            name="single_min_demand") 


{(1, 'closed_front_1', '111', 'col1'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col2'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col3'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col4'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col5'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col6'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col7'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col8'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col9'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col10'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col11'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col12'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_

In [263]:
# Find the optimal solution
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 125 rows, 66 columns and 665 nonzeros
Model fingerprint: 0xda6acfa1
Model has 1 general constraint
Variable types: 6 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 108 rows and 5 columns
Presolve time: 0.00s
Presolved: 17 rows, 61 columns, 125 nonzeros
Variable types: 1 continuous, 60 integer (60 binary)

Root relaxation: objective 1.489842e+01, 49 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   14.89842    0    8   -0.00000   14.89842      -     -    0s
H   

In [264]:
### Post processing output values 
product_flow = pd.DataFrame(columns=["Machine_id", "Machine_type", "Product","Column","Values","Capacity"])
for arc in combinations:
    #if select_single[arc].x > 1e-6:
    product_flow = product_flow.append({"Machine_id": arc[0], "Machine_type": arc[1],"Product": arc[2], "Column": arc[3],"Values": select_single[arc].x,"Capacity": select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]],"dos":select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]]/spdm[arc[0],arc[1],str(arc[2])],"spd":spdm[arc[0],arc[1],str(arc[2])]}, ignore_index=True)  
#product_flow.index=[''] * len(product_flow)
product_flow

,Machine_id,Machine_type,Product,Column,Values,Capacity,dos,spd
0,1,closed_front_1,111,col1,-0.0,-0.0,-0.000000,5.4
1,1,closed_front_1,111,col2,-0.0,-0.0,-0.000000,5.4
2,1,closed_front_1,111,col3,-0.0,-0.0,-0.000000,5.4
3,1,closed_front_1,111,col4,1.0,11.0,2.037037,5.4
4,1,closed_front_1,111,col5,-0.0,-0.0,-0.000000,5.4
5,1,closed_front_1,111,col6,-0.0,-0.0,-0.000000,5.4
6,1,closed_front_1,111,col7,1.0,11.0,2.037037,5.4
7,1,closed_front_1,111,col8,-0.0,-0.0,-0.000000,5.4
8,1,closed_front_1,111,col9,-0.0,-0.0,-0.000000,5.4
9,1,closed_front_1,111,col10,-0.0,-0.0,-0.000000,5.4


In [255]:
spdm

{(1, 'closed_front_1', '111'): 5.4,
 (1, 'closed_front_1', '222'): 1.5,
 (1, 'closed_front_1', '333'): 12.1,
 (1, 'closed_front_1', '444'): 8.6,
 (1, 'closed_front_1', '555'): 12.1}

In [250]:
h=product_flow[['Machine_id','Machine_type','Product'
                ,'Values','Capacity','dos','spd']]
k=h.groupby(['Machine_id','Machine_type','Product','spd','Values'])

In [251]:
k.first()

Capacity       dos
Machine_id Machine_type   Product spd  Values                    
1          closed_front_1 111     5.4  -0.0        -0.0 -0.000000
                                        1.0        11.0  2.037037
                          222     1.5  -0.0        -0.0 -0.000000
                                        1.0        11.0  7.333333
                          333     12.1 -0.0        -0.0 -0.000000
                                        1.0        11.0  0.909091
                          444     8.6  -0.0        -0.0 -0.000000
                                        1.0         9.0  1.046512
                          555     12.1 -0.0        -0.0 -0.000000
                                        1.0         9.0  0.743802

In [265]:
z

<gurobi.Var Z (value 11.157024793388429)>

In [266]:
y

{111: <gurobi.Var y[111] (value 20.370370370370367)>,
 222: <gurobi.Var y[222] (value 36.666666666666664)>,
 333: <gurobi.Var y[333] (value 13.636363636363637)>,
 444: <gurobi.Var y[444] (value 15.697674418604652)>,
 555: <gurobi.Var y[555] (value 11.157024793388429)>}

In [267]:
tmp ={j:3 for j in zip(machines_single['machine_id'],machines_single['machine_type'],machines_single['item_id'],machines_single['Column'])}




In [268]:
tmp()

{(1, 'closed_front_1', '111', 'col1'): 3,
 (1, 'closed_front_1', '111', 'col2'): 3,
 (1, 'closed_front_1', '111', 'col3'): 3,
 (1, 'closed_front_1', '111', 'col4'): 3,
 (1, 'closed_front_1', '111', 'col5'): 3,
 (1, 'closed_front_1', '111', 'col6'): 3,
 (1, 'closed_front_1', '111', 'col7'): 3,
 (1, 'closed_front_1', '111', 'col8'): 3,
 (1, 'closed_front_1', '111', 'col9'): 3,
 (1, 'closed_front_1', '111', 'col10'): 3,
 (1, 'closed_front_1', '111', 'col11'): 3,
 (1, 'closed_front_1', '111', 'col12'): 3,
 (1, 'closed_front_1', '222', 'col1'): 3,
 (1, 'closed_front_1', '222', 'col2'): 3,
 (1, 'closed_front_1', '222', 'col3'): 3,
 (1, 'closed_front_1', '222', 'col4'): 3,
 (1, 'closed_front_1', '222', 'col5'): 3,
 (1, 'closed_front_1', '222', 'col6'): 3,
 (1, 'closed_front_1', '222', 'col7'): 3,
 (1, 'closed_front_1', '222', 'col8'): 3,
 (1, 'closed_front_1', '222', 'col9'): 3,
 (1, 'closed_front_1', '222', 'col10'): 3,
 (1, 'closed_front_1', '222', 'col11'): 3,
 (1, 'closed_front_1', '222',

In [272]:
## Modification in above 
y = model.addVars(products, lb=-GRB.INFINITY, name="y")
z = model.addVar(vtype=GRB.CONTINUOUS, name="Z")
model.addConstrs((y[p] == gp.quicksum(select_single[1,'closed_front_1',str(p),c]*cap['closed_front_1',volp[int(p)],c] for c in columns)/spdm[1,'closed_front_1',str(p)] 
                  for p in products),name="set_y")
model.addConstr(z == min_(y), name="min constraint")
model.setObjective(z, sense=GRB.MAXIMIZE)
               

In [273]:
## Single machine Minimum demand has to be satisfied - Constraint2
model.addConstrs((select_single.sum(m,i,p,'*')*cap[i,volp[int(p)],c] >= spdm[m,i,p]
             for m,i,p,c in select_single.keys()),
            name="single_min_demand") 


{(1, 'closed_front_1', '111', 'col1'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col2'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col3'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col4'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col5'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col6'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col7'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col8'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col9'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col10'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col11'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col12'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_

In [274]:
## Single machine - Constraint2
model.addConstrs((select_single.sum(m,i,'*',c) <= 1
             for m,i,p,c in select_single.keys()),
            name="single_1_col_1prod") 


{(1, 'closed_front_1', '111', 'col1'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col2'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col3'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col4'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col5'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col6'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col7'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col8'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_1', '111', 'col9'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col10'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col11'): <gurobi.Constr *Awaiting Model Update*>,
 (1,
  'closed_front_1',
  '111',
  'col12'): <gurobi.Constr *Awaiting Model Update*>,
 (1, 'closed_front_

In [271]:
select_single

{(1,
  'closed_front_1',
  '111',
  'col1'): <gurobi.Var select2[1,closed_front_1,111,col1] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col2'): <gurobi.Var select2[1,closed_front_1,111,col2] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col3'): <gurobi.Var select2[1,closed_front_1,111,col3] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col4'): <gurobi.Var select2[1,closed_front_1,111,col4] (value 1.0)>,
 (1,
  'closed_front_1',
  '111',
  'col5'): <gurobi.Var select2[1,closed_front_1,111,col5] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col6'): <gurobi.Var select2[1,closed_front_1,111,col6] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col7'): <gurobi.Var select2[1,closed_front_1,111,col7] (value 1.0)>,
 (1,
  'closed_front_1',
  '111',
  'col8'): <gurobi.Var select2[1,closed_front_1,111,col8] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'col9'): <gurobi.Var select2[1,closed_front_1,111,col9] (value -0.0)>,
 (1,
  'closed_front_1',
  '111',
  'co

In [275]:
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 250 rows, 72 columns and 1750 nonzeros
Model fingerprint: 0xa3a702c5
Model has 2 general constraints
Variable types: 12 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [7e-01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]

MIP start from previous solve produced solution with objective 2.2314 (0.01s)
Loaded MIP start from previous solve with objective 2.2314

Presolve removed 229 rows and 7 columns
Presolve time: 0.00s
Presolved: 21 rows, 65 columns, 133 nonzeros
Variable types: 1 continuous, 64 integer (60 binary)

Root relaxation: objective 2.608696e+00, 45 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

In [276]:
y

{111: <gurobi.Var y[111] (value 4.0740740740740735)>,
 222: <gurobi.Var y[222] (value 7.333333333333333)>,
 333: <gurobi.Var y[333] (value 2.7272727272727275)>,
 444: <gurobi.Var y[444] (value 3.1395348837209305)>,
 555: <gurobi.Var y[555] (value 2.231404958677686)>}

In [277]:
z

<gurobi.Var Z (value 2.231404958677686)>

In [301]:
### Post processing output values 
product_flow = pd.DataFrame(columns=["Machine_id", "Machine_type", "Product","Column","Values","Capacity"])
for arc in combinations:
    #if select_single[arc].x > 1e-6:
    product_flow = product_flow.append({"Machine_id": arc[0], "Machine_type": arc[1],"Product": arc[2], "Column": arc[3],"Values": select_single[arc].x,"Capacity": select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]],"dos":select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]]/spdm[arc[0],arc[1],str(arc[2])],"spd":spdm[arc[0],arc[1],str(arc[2])]}, ignore_index=True)  
#product_flow.index=[''] * len(product_flow)
product_flow

,Machine_id,Machine_type,Product,Column,Values,Capacity,dos,spd
0,1,closed_front_1,111,col1,0.0,0.0,0.000000,5.4
1,1,closed_front_1,111,col2,0.0,0.0,0.000000,5.4
2,1,closed_front_1,111,col3,0.0,0.0,0.000000,5.4
3,1,closed_front_1,111,col4,1.0,11.0,2.037037,5.4
4,1,closed_front_1,111,col5,0.0,0.0,0.000000,5.4
5,1,closed_front_1,111,col6,0.0,0.0,0.000000,5.4
6,1,closed_front_1,111,col7,1.0,11.0,2.037037,5.4
7,1,closed_front_1,111,col8,0.0,0.0,0.000000,5.4
8,1,closed_front_1,111,col9,0.0,0.0,0.000000,5.4
9,1,closed_front_1,111,col10,0.0,0.0,0.000000,5.4


In [302]:
column_k=product_flow[product_flow['Values']==1]

In [303]:
column_k## one solution

,Machine_id,Machine_type,Product,Column,Values,Capacity,dos,spd
3,1,closed_front_1,111,col4,1.0,11.0,2.037037,5.4
6,1,closed_front_1,111,col7,1.0,11.0,2.037037,5.4
22,1,closed_front_1,222,col11,1.0,11.0,7.333333,1.5
25,1,closed_front_1,333,col2,1.0,11.0,0.909091,12.1
26,1,closed_front_1,333,col3,1.0,11.0,0.909091,12.1
33,1,closed_front_1,333,col10,1.0,11.0,0.909091,12.1
36,1,closed_front_1,444,col1,1.0,9.0,1.046512,8.6
41,1,closed_front_1,444,col6,1.0,9.0,1.046512,8.6
47,1,closed_front_1,444,col12,1.0,9.0,1.046512,8.6
52,1,closed_front_1,555,col5,1.0,9.0,0.743802,12.1


In [300]:
k

,Machine_id,Machine_type,Product,Capacity,dos,spd
3,1,closed_front_1,111,11.0,2.037037,5.4
6,1,closed_front_1,111,11.0,2.037037,5.4
22,1,closed_front_1,222,11.0,7.333333,1.5
25,1,closed_front_1,333,11.0,0.909091,12.1
26,1,closed_front_1,333,11.0,0.909091,12.1
33,1,closed_front_1,333,11.0,0.909091,12.1
36,1,closed_front_1,444,9.0,1.046512,8.6
41,1,closed_front_1,444,9.0,1.046512,8.6
47,1,closed_front_1,444,9.0,1.046512,8.6
52,1,closed_front_1,555,9.0,0.743802,12.1


In [299]:
k=column_k.drop(['Column','Values'], axis=1)

In [285]:
k

,Machine_id,Machine_type,Product,Capacity,dos,spd
3,1,closed_front_1,111,11.0,2.037037,5.4
6,1,closed_front_1,111,11.0,2.037037,5.4
22,1,closed_front_1,222,11.0,7.333333,1.5
25,1,closed_front_1,333,11.0,0.909091,12.1
26,1,closed_front_1,333,11.0,0.909091,12.1
33,1,closed_front_1,333,11.0,0.909091,12.1
36,1,closed_front_1,444,9.0,1.046512,8.6
41,1,closed_front_1,444,9.0,1.046512,8.6
47,1,closed_front_1,444,9.0,1.046512,8.6
52,1,closed_front_1,555,9.0,0.743802,12.1


In [308]:
j=k.groupby(['Machine_id','Machine_type','Product']).agg(sum)

In [309]:
j

Capacity       dos   spd
Machine_id Machine_type   Product                          
1          closed_front_1 111          22.0  4.074074  10.8
                          222          11.0  7.333333   1.5
                          333          33.0  2.727273  36.3
                          444          27.0  3.139535  25.8
                          555          27.0  2.231405  36.3

In [322]:
j['spd']=spdm.values()

In [323]:
j ## sencond solution

Capacity       dos   spd
Machine_id Machine_type   Product                          
1          closed_front_1 111          22.0  4.074074   5.4
                          222          11.0  7.333333   1.5
                          333          33.0  2.727273  12.1
                          444          27.0  3.139535   8.6
                          555          27.0  2.231405  12.1

In [297]:
z

<gurobi.Var Z (value 2.231404958677686)>

In [ ]:
machine_id	machine_type	col1	col2	col3	col4	col5	col6	col7	col8	col9	col10	col11	col12
1	closed_front_1	111	222	333	444	555	111	222	333	444	555	111	222